# FFT
El algoritmo para expresar los datos discretoscomo ondas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from numpy.fft import *
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")

In [ ]:
# aqui el filtrado, donde se utiliza el correspondiente a las librerias que ya estan adaptadas en C
# para funcionar mas rapido
def filter_signal(signal, limite=1e8):
    # real fast fourier transform, asume data real, y explota la simetria
    # ahorrandose calculo de la mitad de las señales
    fourier = rfft(signal)
    # d: sampling frecuency, corresponde a los 20ms / data total
    frequencias = rfftfreq(signal.size, d=20e-3/signal.size)
    # igualamos a 0 en lugar de eliminar, para manener la shape
    fourier[frequencias > limite] = 0
    return irfft(fourier)

print('Para nuestros datos, el spacing entre frecuencias debe de ser:',20e-3/signals[0,0,:].size)

In [ ]:
limite = 1e3
filtered = filter_signal(signals[0, 0, :], limite)

plt.figure(figsize=(15, 10))
plt.plot(signals[0, 0, :], label='Original')
plt.plot(filtered, label='Filtrada')
plt.legend()
plt.title(f"FFT Denoising limite = {limite}", size=15)
plt.show()

algortimo FFT

In [ ]:
def FFT(P: np.array):
    N = len(P)

    if N == 1:
        return P

    if (N%2 != 0): # es innecesario pero ayuda al lector
        P = np.insert(P, N-1,0)
    
    w = np.exp(2 * np.pi * 1j / N)
    Ppar, Pimpar = P[0::2], P[1::2] # coefciientes polinomio
    y_par, y_impar = FFT(Ppar), FFT(Pimpar)
    y = [0] * N

    

    for j in range(N//2):
        y[j] = y_par[j] + w**j * y_impar[j]
        y[j+ N//2] = y_par[j] - w**j * y_impar[j]
    return y

def IFFT(P: np.array):
    N = len(P)

    if N == 1:
        return P
    w = (1/N) * np.exp(-2 * np.pi * 1j / N)
    Ppar, Pimpar = P[0::2], P[1::2] # coefciientes polinomio
    y_par, y_impar = IFFT(Ppar), IFFT(Pimpar)
    y = [0] * N
    for j in range(N/2):
        y[j] = y_par[j] + w**j * y_impar[j]
        y[j+ N/2] = y_par[j] - w**j * y_impar[j]
    return y